In [51]:
import pickle
import pandas as pd
import numpy as np
from decimal import Decimal
from decimal import *
df = pd.read_pickle(r'experiment.p')

In [3]:
df

,id_indexer,pool_delegated_stake,pool_locked_stake,shares,delegators,GRT,indexing_revenue,query_revenue,simulation,subset,run,substep,timestep
5,indexer,1243612,0,1243612,{'indexer': <model.parts.delegator.Delegator o...,0E-60,0,0,0,0,1,5,1
10,indexer,1243612,0,1243612,{'indexer': <model.parts.delegator.Delegator o...,0E-60,0,0,0,0,1,5,2
15,indexer,1243612,0,1243612,{'indexer': <model.parts.delegator.Delegator o...,0E-60,0,0,0,0,1,5,3
20,indexer,1243612,0,1243612,{'indexer': <model.parts.delegator.Delegator o...,0E-60,0,0,0,0,1,5,4
25,indexer,1243612,0,1243612,{'indexer': <model.parts.delegator.Delegator o...,0E-60,0,0,0,0,1,5,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
430,indexer,1322396.969876315547710680795,0,1319719.496804887487388932476,{'indexer': <model.parts.delegator.Delegator o...,2479579.061618249999991237460,2480000.000000000000000000,0.970125109574860079,0,0,1,5,86
435,indexer,1322396.969876315547710680795,0,1319719.496804887487388932476,{'indexer': <model.parts.delegator.Delegator o...,2479579.061618249999991237460,2480000.000000000000000000,0.970125109574860079,0,0,1,5,87
440,indexer,1322396.969876315547710680795,0,1319719.496804887487388932476,{'indexer': <model.parts.delegator.Delegator o...,2479579.061618249999991237460,2480000.000000000000000000,0.970125109574860079,0,0,1,5,88
445,indexer,1322396.969876315547710680795,0,1319719.496804887487388932476,{'indexer': <model.parts.delegator.Delegator o...,2479579.061618249999991237460,2480000.000000000000000000,0.970125109574860079,0,0,1,5,89


In [5]:
df.reset_index(inplace = True)

In [6]:
df

,index,id_indexer,pool_delegated_stake,pool_locked_stake,shares,delegators,GRT,indexing_revenue,query_revenue,simulation,subset,run,substep,timestep
0,5,indexer,1243612,0,1243612,{'indexer': <model.parts.delegator.Delegator o...,0E-60,0,0,0,0,1,5,1
1,10,indexer,1243612,0,1243612,{'indexer': <model.parts.delegator.Delegator o...,0E-60,0,0,0,0,1,5,2
2,15,indexer,1243612,0,1243612,{'indexer': <model.parts.delegator.Delegator o...,0E-60,0,0,0,0,1,5,3
3,20,indexer,1243612,0,1243612,{'indexer': <model.parts.delegator.Delegator o...,0E-60,0,0,0,0,1,5,4
4,25,indexer,1243612,0,1243612,{'indexer': <model.parts.delegator.Delegator o...,0E-60,0,0,0,0,1,5,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,430,indexer,1322396.969876315547710680795,0,1319719.496804887487388932476,{'indexer': <model.parts.delegator.Delegator o...,2479579.061618249999991237460,2480000.000000000000000000,0.970125109574860079,0,0,1,5,86
86,435,indexer,1322396.969876315547710680795,0,1319719.496804887487388932476,{'indexer': <model.parts.delegator.Delegator o...,2479579.061618249999991237460,2480000.000000000000000000,0.970125109574860079,0,0,1,5,87
87,440,indexer,1322396.969876315547710680795,0,1319719.496804887487388932476,{'indexer': <model.parts.delegator.Delegator o...,2479579.061618249999991237460,2480000.000000000000000000,0.970125109574860079,0,0,1,5,88
88,445,indexer,1322396.969876315547710680795,0,1319719.496804887487388932476,{'indexer': <model.parts.delegator.Delegator o...,2479579.061618249999991237460,2480000.000000000000000000,0.970125109574860079,0,0,1,5,89


In [21]:
df.iloc[7].delegators['0x63f8168f665f79101a0e0da2313cd1391e1c5e27'].shares

Decimal('1088.074676093251638179797803')

In [38]:
def decimal_from_value(value):
    return Decimal(value)

In [43]:
indexer_df = pd.read_csv('GraphQL_data/singleIndexer.csv')
indexer_df['epoch'] = np.floor((indexer_df['blockNumber'] - 11474307)/6500)
# scale to decimal
indexer_df['shares'] = indexer_df['shares'].apply(lambda x: decimal_from_value(x))

In [54]:
if indexer_df.iloc[1]['shares'] * Decimal(10 ** -18) == df.iloc[7].delegators['0x63f8168f665f79101a0e0da2313cd1391e1c5e27'].shares:
    print('Good')
else:
    print(f"{indexer_df.iloc[1]['shares'] * Decimal(10 ** -18)}, {df.iloc[7].delegators['0x63f8168f665f79101a0e0da2313cd1391e1c5e27'].shares}") 

1094.500000000000078303183128, 1088.074676093251638179797803
